In [1]:
import pandas as pd
import numpy as np

In [2]:
anime = pd.read_csv('anime.csv')
usersD = pd.read_csv('usersD.csv')
usersS = pd.read_csv('usersS.csv')

In [3]:
anime.columns

Index(['anime_id', 'Name', 'English name', 'Other name', 'Score', 'Genres',
       'Synopsis', 'Type', 'Episodes', 'Aired', 'Premiered', 'Status',
       'Producers', 'Licensors', 'Studios', 'Source', 'Duration', 'Rating',
       'Rank', 'Popularity', 'Favorites', 'Scored By', 'Members', 'Image URL'],
      dtype='object')

In [4]:
anime['Image URL'][1]

'https://cdn.myanimelist.net/images/anime/1439/93480.jpg'

In [5]:
usersD.head()

,Mal ID,Username,Gender,Birthday,Location,Joined,Days Watched,Mean Score,Watching,Completed,On Hold,Dropped,Plan to Watch,Total Entries,Rewatched,Episodes Watched
0,1,Xinil,Male,1985-03-04T00:00:00+00:00,California,2004-11-05T00:00:00+00:00,142.3,7.37,1.0,233.0,8.0,93.0,64.0,399.0,60.0,8458.0
1,3,Aokaado,Male,NaN,"Oslo, Norway",2004-11-11T00:00:00+00:00,68.6,7.34,23.0,137.0,99.0,44.0,40.0,343.0,15.0,4072.0
2,4,Crystal,Female,NaN,"Melbourne, Australia",2004-11-13T00:00:00+00:00,212.8,6.68,16.0,636.0,303.0,0.0,45.0,1000.0,10.0,12781.0
3,9,Arcane,NaN,NaN,NaN,2004-12-05T00:00:00+00:00,30.0,7.71,5.0,54.0,4.0,3.0,0.0,66.0,0.0,1817.0
4,18,Mad,NaN,NaN,NaN,2005-01-03T00:00:00+00:00,52.0,6.27,1.0,114.0,10.0,5.0,23.0,153.0,42.0,3038.0


In [6]:
usersS.head()

,user_id,Username,anime_id,Anime Title,rating
0,1,Xinil,21,One Piece,9
1,1,Xinil,48,.hack//Sign,7
2,1,Xinil,320,A Kite,5
3,1,Xinil,49,Aa! Megami-sama!,8
4,1,Xinil,304,Aa! Megami-sama! Movie,8


In [7]:
usersD.columns

Index(['Mal ID', 'Username', 'Gender', 'Birthday', 'Location', 'Joined',
       'Days Watched', 'Mean Score', 'Watching', 'Completed', 'On Hold',
       'Dropped', 'Plan to Watch', 'Total Entries', 'Rewatched',
       'Episodes Watched'],
      dtype='object')

In [8]:
usersS.columns

Index(['user_id', 'Username', 'anime_id', 'Anime Title', 'rating'], dtype='object')

In [9]:
anime.shape

(24905, 24)

In [10]:
usersD.shape

(731290, 16)

In [11]:
usersS.shape

(24325191, 5)

In [12]:
anime.isnull().sum()

anime_id        0
Name            0
English name    0
Other name      0
Score           0
Genres          0
Synopsis        0
Type            0
Episodes        0
Aired           0
Premiered       0
Status          0
Producers       0
Licensors       0
Studios         0
Source          0
Duration        0
Rating          0
Rank            0
Popularity      0
Favorites       0
Scored By       0
Members         0
Image URL       0
dtype: int64

In [13]:
usersD.duplicated().sum()

0

In [14]:
usersS.duplicated().sum()

0

In [15]:
anime.duplicated().sum()

0

In [16]:
def get_anime_image(anime_id):
    try:
        row = anime.loc[anime['anime_id'] == anime_id]
        if not row.empty:
            url = row.iloc[0].get('Image URL', None)
            if isinstance(url, str) and len(url.strip()) > 5:
                return url
        # if not found or invalid, try by index lookup safeguard
    except Exception:
        pass
    # final fallback (good-looking dark placeholder)
    return 'https://via.placeholder.com/280x140/1a1a2e/ffffff?text=No+Image'

In [17]:
def coerce_numeric_rating(frame, source_col_candidates):
    for col in source_col_candidates:
        if col in frame.columns:
            # if it's a string sentiment map, handle outside; otherwise try numeric
            if pd.api.types.is_numeric_dtype(frame[col]):
                return frame[col]
            # try to coerce
            try:
                return pd.to_numeric(frame[col], errors='coerce')
            except Exception:
                pass
    # if nothing works, return all NaN series with same length
    return pd.Series([np.nan] * len(frame), index=frame.index)

 Popularity Based

In [18]:
recommender_with_name = anime.copy()

In [19]:
print("Before merge:", len(anime), len(anime))
print("After merge:", len(recommender_with_name))

Before merge: 24905 24905
After merge: 24905


In [20]:
print(recommender_with_name.head())

   anime_id                             Name             English name  \
0         1                     Cowboy Bebop             Cowboy Bebop   
1         5  Cowboy Bebop: Tengoku no Tobira  Cowboy Bebop: The Movie   
2         6                           Trigun                   Trigun   
3         7               Witch Hunter Robin       Witch Hunter Robin   
4         8                   Bouken Ou Beet   Beet the Vandel Buster   

                         Other name Score  \
0                         カウボーイビバップ  8.75   
1                    カウボーイビバップ 天国の扉  8.38   
2                             トライガン  8.22   
3  Witch Hunter ROBIN (ウイッチハンターロビン)  7.25   
4                            冒険王ビィト  6.94   

                                 Genres  \
0         Action, Award Winning, Sci-Fi   
1                        Action, Sci-Fi   
2             Action, Adventure, Sci-Fi   
3  Action, Drama, Mystery, Supernatural   
4      Adventure, Fantasy, Supernatural   

                               

In [34]:
if 'Rating' in anime.columns and 'Popularity' in anime.columns:
    # Fill missing ratings with mean
    anime['Rating'] = pd.to_numeric(anime['Rating'], errors='coerce')
    anime['Rating'] = anime['Rating'].fillna(anime['Rating'].mean())
    
    # Create weighted score (combining rating and popularity)
    anime['weighted_score'] = anime['Rating'] * 0.7 + (1 - anime['Popularity']/anime['Popularity'].max()) * 0.3
    
    # Get top recommendations
    animes_trimmed = anime[['anime_id', 'Name', 'Popularity', 'Rating', 'weighted_score']].copy()
    animes_trimmed = animes_trimmed.sort_values('weighted_score', ascending=False)
    
    print("\nTop 10 Popular Anime:")
    print(animes_trimmed.head(10)[['Name', 'Rating', 'Popularity']])
else:
    animes_trimmed = anime[['anime_id', 'Name']].copy()
    print("Rating/Popularity columns not found, using basic trimmed dataset")


Top 10 Popular Anime:
                              Name  Rating  Popularity
0                     Cowboy Bebop     NaN          43
1  Cowboy Bebop: Tengoku no Tobira     NaN         602
2                           Trigun     NaN         246
3               Witch Hunter Robin     NaN        1795
4                   Bouken Ou Beet     NaN        5126
5                     Eyeshield 21     NaN        1252
6             Hachimitsu to Clover     NaN         862
7       Hungry Heart: Wild Striker     NaN        4212
8           Initial D Fourth Stage     NaN        1273
9                          Monster     NaN         142


In [35]:
def get_popular_recommendations(n=10):
    """Get top N popular anime recommendations"""
    if 'weighted_score' in anime.columns:
        top_anime = anime.nlargest(n, 'weighted_score')
        return top_anime[['Name', 'Rating', 'Popularity', 'anime_id']].reset_index(drop=True)
    else:
        return anime[['Name', 'anime_id']].head(n).reset_index(drop=True)

CONTENT BASED RECOMMENDER

In [36]:
def create_content_recommender():
    """Create content-based recommendation system"""
    if 'Genres' in anime.columns:
        # Handle missing genres
        anime['Genres'] = anime['Genres'].fillna('Unknown')
        
        # Create TF-IDF matrix for genres
        tfidf = TfidfVectorizer(stop_words='english', lowercase=True)
        tfidf_matrix = tfidf.fit_transform(anime['Genres'])
        
        # Calculate cosine similarity
        cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
        
        return cosine_sim, anime.reset_index(drop=True)
    else:
        print("Genres column not found, content-based filtering unavailable")
        return None, anime

In [37]:
def get_content_recommendations(anime_name, cosine_sim, n=10):
    """Get content-based recommendations for a given anime"""
    if cosine_sim is None:
        return "Content-based recommendations not available"
    
    try:
        # Get the index of the anime
        idx = anime[anime['Name'].str.contains(anime_name, case=False, na=False)].index[0]
        
        # Get similarity scores
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # Get top similar anime (excluding itself)
        sim_scores = sim_scores[1:n+1]
        anime_indices = [i[0] for i in sim_scores]
        
        return anime.iloc[anime_indices][['Name', 'Rating', 'Genres']].reset_index(drop=True)
    except IndexError:
        return f"Anime '{anime_name}' not found in database"

COLLABORATIVE FILTERING

In [38]:
print("usersS columns:", usersS.columns.tolist())

usersS columns: ['user_id', 'Username', 'anime_id', 'Anime Title', 'rating']


In [39]:
print(usersS.head())

   user_id Username  anime_id             Anime Title  rating
0        1    Xinil        21               One Piece       9
1        1    Xinil        48             .hack//Sign       7
2        1    Xinil       320                  A Kite       5
3        1    Xinil        49        Aa! Megami-sama!       8
4        1    Xinil       304  Aa! Megami-sama! Movie       8


In [40]:
if 'Anime Title' in usersS.columns:
    user_anime_ratings = usersS.copy()
    user_anime_ratings['Name'] = user_anime_ratings['Anime Title']
else:
    user_anime_ratings = usersS.copy()
    if 'anime_id' in user_anime_ratings.columns and 'anime_id' in anime.columns and 'Name' in anime.columns:
        user_anime_ratings = user_anime_ratings.merge(
            anime[['anime_id','Name']], on='anime_id', how='left'
        )
    else:
        # hard fallback: create a Name column to avoid crashes
        user_anime_ratings['Name'] = user_anime_ratings.get('Name', 'Unknown')

In [41]:
if 'rating' not in user_anime_ratings.columns:
    for alt in ['my_rating', 'score', 'Rating', 'user_rating']:
        if alt in user_anime_ratings.columns:
            user_anime_ratings['rating'] = user_anime_ratings[alt]
            break
# final guard: if missing, fill with NaN so dropna below works
if 'rating' not in user_anime_ratings.columns:
    user_anime_ratings['rating'] = np.nan

In [42]:
a = user_anime_ratings.dropna(subset=['rating']).groupby('anime_id').count()['rating'] > 10
cartoon = a[a].index

In [43]:
filtered_rating = user_anime_ratings[user_anime_ratings['anime_id'].isin(cartoon)]

In [44]:
b = filtered_rating.groupby('user_id').count()['rating'] >= 5
famous_cartoon = b[b].index

In [45]:
final_filtered = filtered_rating[filtered_rating['user_id'].isin(famous_cartoon)]

In [46]:
popular_cartoon = final_filtered.groupby('Name').size()
popular_cartoon = popular_cartoon[popular_cartoon > 50].index

In [47]:
small_data = final_filtered[final_filtered['Name'].isin(popular_cartoon)]

In [59]:
if small_data['Name'].nunique() < 20:
    a = user_anime_ratings.dropna(subset=['rating']).groupby('anime_id').count()['rating'] > 5
    cartoon = a[a].index
    filtered_rating = user_anime_ratings[user_anime_ratings['anime_id'].isin(cartoon)]

    b = filtered_rating.groupby('user_id').count()['rating'] >= 3
    famous_cartoon = b[b].index
    final_filtered = filtered_rating[filtered_rating['user_id'].isin(famous_cartoon)]

    popular_cartoon = final_filtered.groupby('Name').size()
    popular_cartoon = popular_cartoon[popular_cartoon > 20].index
    small_data = final_filtered[final_filtered['Name'].isin(popular_cartoon)]

In [49]:
pt = small_data.pivot_table(
    columns='Name',
    values='rating',
    index='user_id',
).fillna(0)

In [50]:
pt

Name,"""0""","""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi","""Bungaku Shoujo"" Memoire","""Bungaku Shoujo"" Movie","""Eiji""","""Eiyuu"" Kaitai","""Oshi no Ko""",** Kouhai,.hack//G.U. Returner,.hack//G.U. Trilogy,...,s.CRY.ed,s.CRY.ed Alteration I: Tao,s.CRY.ed Alteration II: Quan,xxxHOLiC,xxxHOLiC Movie: Manatsu no Yoru no Yume,xxxHOLiC Rou,xxxHOLiC Shunmuki,xxxHOLiC◆Kei,ēlDLIVE,◯
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1291049,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1291057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1291079,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

In [52]:
if pt.shape[1] == 0:
    # Avoid crash if empty; create a dummy column so code paths still work
    pt = pd.DataFrame({ 'DUMMY': pd.Series(0, index=pd.Index([], name='user_id')) }).T

In [53]:
PC_sparse = csr_matrix(pt.values)
similarity_scores = cosine_similarity(PC_sparse.T, dense_output=False)
similarity_scores.shape
similarity = similarity_scores

In [54]:
import IPython.display as display
from IPython.core.display import HTML

In [55]:
def recommend(anime_name, top_n=12):
    # Check if anime exists in pivot table
    if anime_name not in pt.columns:
        # Try to find similar names if exact match fails
        similar_names = [name for name in pt.columns if anime_name.lower() in name.lower()]
        if similar_names:
            print(f"Exact match not found. Did you mean: {similar_names[:3]}")
            anime_name = similar_names[0]
            print(f"Using: {anime_name}")
        else:
            print(f"Anime '{anime_name}' not found in dataset.")
            print("Available anime names in dataset:")
            print(pt.columns[:10].tolist())
            return
    
    # Get similarity scores for the input anime
    anime_idx = list(pt.columns).index(anime_name)
    sim_vec = similarity_scores[anime_idx]
    # sim_vec is sparse if dense_output=False; ensure dense vector
    sim_scores = sim_vec.toarray().ravel() if hasattr(sim_vec, "toarray") else np.asarray(sim_vec).ravel()
    
    # Get indices of most similar animes (excluding the input anime itself)
    top_n = int(top_n) if top_n is not None else 12
    similar_indices = np.argsort(sim_scores)[::-1]
    similar_indices = [i for i in similar_indices if i != anime_idx][:top_n]
    recommended_Name = [pt.columns[i] for i in similar_indices]
    
    # Build recommendations with proper data handling
    recommendations_data = []
    for Name in recommended_Name:
        # Get anime info from original dataframe
        anime_match = anime[anime['Name'] == Name]
        if not anime_match.empty:
            anime_info = anime_match.iloc[0]
            
            # Calculate average rating from collaborative filtering data
            rating_matches = final_filtered[final_filtered['Name'] == Name]['rating']
            if not rating_matches.empty:
                avg_rating = rating_matches.mean()
                rating = round(float(avg_rating), 1)
            else:
                # Fallback to original numeric score if available
                if 'Score' in anime_info and pd.notna(anime_info['Score']):
                    rating = anime_info['Score']
                elif 'Popularity' in anime_info and pd.api.types.is_number(anime_info['Popularity']):
                    rating = anime_info['Popularity']
                else:
                    rating = 'N/A'
            
            # Generate unique image URL for each anime
            image_url = get_anime_image(anime_info['anime_id'])
                
            recommendations_data.append({
                'Name': Name,
                'Image URL': image_url,
                'rating': rating,
                'anime_id': anime_info['anime_id']
            })

    # Create HTML display with unique images
    cards_html = """
    <div style='display:flex; flex-wrap:wrap; gap:15px; justify-content:center; 
                background:#0a0a0a; padding:20px; border-radius:15px;'>
    <h2 style='width:100%; text-align:center; color:#ffffff; margin-bottom:20px; 
               font-family:Arial,sans-serif;'>Animes Similar to {}</h2>
    """.format(anime_name)
    
    for anime_item in recommendations_data:
        cards_html += f"""
        <div style="width:280px; border-radius:15px; overflow:hidden;
                    box-shadow:0px 6px 15px rgba(0,0,0,0.4); 
                    background:linear-gradient(145deg, #1a1a2e, #16213e); 
                    color:white; text-align:center; padding:15px;
                    transition:transform 0.3s ease, box-shadow 0.3s ease;
                    border:1px solid #333;">
            <img src="{anime_item['Image URL']}" 
                 style="width:100%; height:140px; object-fit:cover; 
                        border-radius:10px; margin-bottom:10px;
                        border:2px solid #444;"
                 onerror="this.src='https://via.placeholder.com/280x140/1a1a2e/ffffff?text=No+Image'"/>
            <h3 style="margin:10px 0; font-size:16px; color:#ffffff; 
                       font-family:Arial,sans-serif; line-height:1.3;
                       min-height:40px; display:flex; align-items:center; 
                       justify-content:center;">{anime_item['Name']}</h3>
            <div style="background:rgba(255,255,255,0.1); padding:8px; 
                        border-radius:8px; margin-top:10px;">
                <p style="margin:0; color:#4CAF50; font-weight:bold; 
                          font-size:14px;">Rating: {anime_item['rating']}</p>
            </div>
        </div>
        """
    cards_html += "</div>"
    
    display.display(HTML(cards_html))

In [58]:
recommend('Death Note')

In [61]:
import pickle
pickle.dump(popular_cartoon,open('popular.PC','wb'))

In [62]:
pickle.dump(pt,open('pt.pa','wb'))
pickle.dump(anime,open('anime.awc','wb'))
pickle.dump(similarity_scores,open('similarity_scores.','wb'))